<a href="https://colab.research.google.com/github/jwells52/creating-ai-enabled-systems/blob/main/Research%20Project/notebooks/fsl_experiment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install easyfsl

In [1]:
%pip install easyfsl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 1.8 MB/s eta 0:00:00


### Download Humpback Whale Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -rf /root/.kaggle && mkdir /root/.kaggle && cp /content/drive/MyDrive/Research-Project/kaggle.json /root/.kaggle/kaggle.json && chmod 600 /root/.kaggle/kaggle.json && kaggle competitions download -c humpback-whale-identification
!unzip humpback-whale-identification.zip

### Clone GitHub Repo

In [4]:
!git clone https://github.com/jwells52/creating-ai-enabled-systems.git
%cd creating-ai-enabled-systems/Research\ Project

Cloning into 'creating-ai-enabled-systems'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 193 (delta 0), reused 0 (delta 0), pack-reused 187
Receiving objects: 100% (193/193), 157.61 MiB | 13.18 MiB/s, done.
Resolving deltas: 100% (79/79), done.
Updating files: 100% (40/40), done.
/content/creating-ai-enabled-systems/Research Project


### Imports

In [5]:
import os

import torch

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

from easyfsl.methods import PrototypicalNetworks, FewShotClassifier, SimpleShot
from easyfsl.utils import evaluate
from easyfsl.samplers import TaskSampler

from torch import Tensor, nn
from torch.optim import SGD, Optimizer, Adam
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet18, resnet34, resnet152

from typing import Callable

from modules.data_utils import HumpbackWhaleDataset, remove_new_whale_class, create_loaders
from modules.train import train_fsl, device, transform

### Look at classes that have low performance

In [6]:
# Load into dataset
# For each class, calculate accuracy
# Then plot barplot of accuracy for each class
# Look at the characteristics of images for the classes that have low performance

In [7]:
df = remove_new_whale_class(
  pd.read_csv('/content/creating-ai-enabled-systems/Research Project/data/images_and_ids.csv')
)


df = df[df['class_count'] > 8]

In [8]:
n_query = 5
n_shot  = 3
n_way   = 5

In [18]:
cnn = resnet34().to(device)
few_shot_classifier = PrototypicalNetworks(cnn).to(device)
few_shot_classifier.load_state_dict(
  torch.load('/content/creating-ai-enabled-systems/Research Project/models/prototypical_network_resnet34_prod_last_epoch')
)

TypeError: ignored

In [14]:
few_shot_classifier.eval()

AttributeError: ignored

In [13]:
print(few_shot_classifier)

<bound method Module.state_dict of PrototypicalNetworks(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64

In [ ]:
class_performances = dict()
for id, df_id in df.groupby('Id'):
  dataset_id = HumpbackWhaleDataset('/content/train', df_id, transform=transform)

  sampler = TaskSampler(
      dataset_id, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=10
  )

  loader = DataLoader(
      dataset_id,
      batch_sampler=sampler,
      num_workers=12,
      pin_memory=True,
      collate_fn=sampler.episodic_collate_fn
  )

  acc = evaluate(
    model, loader, device=device, tqdm_prefix="Validation"
  )

,Image,Id,class_count
2829,2f31725c6.jpg,w_0027efa,10
2929,30eac8c9f.jpg,w_0027efa,10
3592,3c4235ad2.jpg,w_0027efa,10
4335,47bd1d3bb.jpg,w_0027efa,10
5349,5934e01ea.jpg,w_0027efa,10
7501,7b9a5e4d8.jpg,w_0027efa,10
10962,b2acbd8df.jpg,w_0027efa,10
12090,c5658abf0.jpg,w_0027efa,10
12553,cc6c1a235.jpg,w_0027efa,10
14654,ee87a2369.jpg,w_0027efa,10
